In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.drop(['RowNumber', 'CustomerId','Surname'], axis=1, inplace=True)

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
LE_gender = LabelEncoder()
data.Gender = LE_gender.fit_transform(data.Gender)

In [7]:
OHE_geo = OneHotEncoder(sparse_output=False)

In [8]:
geo_coded = OHE_geo.fit_transform(data[['Geography']])

In [9]:
OHE_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [14]:
geo_coded_df = pd.DataFrame(geo_coded, columns=OHE_geo.get_feature_names_out())

In [15]:
geo_coded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [16]:
data1 = pd.concat([data.drop(['Geography'],axis=1), geo_coded_df], axis=1)

In [17]:
data1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(LE_gender, file)
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(OHE_geo, file)

In [19]:
x = data1.drop(['Exited'],axis=1)
y = data1['Exited']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [20]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [22]:
x_train

array([[ 1.78974378,  0.91255717,  0.94394225, ..., -1.00551521,
         1.73436329, -0.57388614],
       [ 0.26040483, -1.09582175,  2.35932486, ..., -1.00551521,
        -0.57658047,  1.74250594],
       [-0.22526362,  0.91255717, -0.47144036, ..., -1.00551521,
        -0.57658047,  1.74250594],
       ...,
       [-1.97160337, -1.09582175,  0.09471269, ..., -1.00551521,
         1.73436329, -0.57388614],
       [ 0.60140608, -1.09582175, -0.47144036, ...,  0.99451504,
        -0.57658047, -0.57388614],
       [ 0.21907135,  0.91255717,  0.09471269, ..., -1.00551521,
         1.73436329, -0.57388614]], shape=(8000, 12))

In [23]:
with open('standard_scaler_x.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [28]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)), # HL1 #Connected with input layer
    Dense(32, activation='relu'), #HL2
    Dense(1, activation='sigmoid') #Output layer
])

C:\Users\2424153\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

opt = Adam(learning_rate=0.1)
loss = BinaryCrossentropy()

In [35]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [37]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [38]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs = 100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.7964 - loss: 0.4673 - val_accuracy: 0.7890 - val_loss: 0.4312
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8033 - loss: 0.4216 - val_accuracy: 0.8080 - val_loss: 0.4172
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8039 - loss: 0.4201 - val_accuracy: 0.8210 - val_loss: 0.4210
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8033 - loss: 0.4130 - val_accuracy: 0.8355 - val_loss: 0.3963
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.7975 - loss: 0.4399 - val_accuracy: 0.7890 - val_loss: 0.4930
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7985 - loss: 0.4272 - val_accuracy: 0.7890 - val_loss: 0.4361
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8235 - loss: 0.4035 - val_accuracy: 0.8050 - val_loss: 0.4095
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8295 - loss: 0.3910 -

In [39]:
model.save('model.h5')

In [41]:
%load_ext tensorboard

In [43]:
%tensorboard --logdir logs/fit